In [25]:
import pandas as pd
import re
registro = ""
palavras = []
with open("../corpus/Bosque_CF_7.4_cgd.txt", "r", encoding = "ISO-8859-1") as file:
    for line in file:
        
        if len(registro) > 0:
            palavras.append(line)
        
        if line.find("id=\"2") >= 0:
            registro = line
        
        if len(registro) > 0 and line.find("</s>") >= 0:
            break
            
registro

textRegex = re.compile(r'text=\"([^\"]+)\"')
frase = textRegex.findall(registro)[0]
frase

'BRASÍLIA Pesquisa Datafolha publicada hoje revela um dado supreendente: recusando uma postura radical, a esmagadora maioria (77%) dos eleitores quer o PT participando do Governo Fernando Henrique Cardoso.'

In [26]:
import ufal.udpipe
# ufal.udpipe.Model etc. are SWIG-magic and cannot be detected by pylint
# pylint: disable=no-member

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [27]:
model = Model('portuguese-br-ud-2.0-conll17-170315.udpipe')
sentences = model.tokenize(frase)

In [28]:
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")
lista = conllu.split('#')[4].split('\n')[1:-2]
table = []
columns = ['Id','Form','Lemma','UPosTag','XPosTag','Feats','Head','DepRel','Deps','Misc']
for li in lista:
    table.append(li.split('\t'))

pd.DataFrame(data=table, columns=columns)

,Id,Form,Lemma,UPosTag,XPosTag,Feats,Head,DepRel,Deps,Misc
0,1,BRASÍLIA,_,PROPN,PNOUN,_,4,nsubj,_,_
1,2,Pesquisa,_,PROPN,PNOUN,_,1,flat,_,_
2,3,Datafolha,_,PROPN,PNOUN,_,1,flat,_,_
3,4,publicada,_,VERB,VERB,_,0,root,_,_
4,5,hoje,_,ADV,ADV,_,4,advmod,_,_
5,6,revela,_,VERB,VERB,_,4,advcl,_,_
6,7,um,_,DET,DET,_,8,det,_,_
7,8,dado,_,NOUN,NOUN,_,6,obj,_,_
8,9,supreendente,_,ADJ,ADJ,_,8,amod,_,SpaceAfter=No
9,10,:,_,PUNCT,.,_,6,punct,_,_
